'''
THIS FILE IS FOR TRAINING THE TREE PHENOLOGY MODEL
CODE WRITTEN FOR CUBESAT MANTIS PROGRAM (DSS)
AUTHOR: CHAZ DAVIES
''' 
Unfinished Steps 
1. Create confusion matrix
2. Switch to Anikets pre-processing code

In [47]:

import tensorflow as tf
import numpy as np
import matplotlib as img 
import rasterio
import os
from rasterio.plot import show
import json
import keras
import import_ipynb
import sys
from sklearn.model_selection import train_test_split


In [26]:
sys.path.append('..')
from preprocessing.Image_Process_Model.load_and_convert import loadDataSet, convertImages, convertImage

In [27]:

path = 'C:\\Users\\chazd\\OneDrive\\Desktop\\git\\DSS\\DSS-MANTIS-OBC-AI\\Images\\algal-bloom\\Lake_torment_HAB_2016-2022_10m'


In [28]:
# Make array to store label names

labels = {
        "no-HAB-present" : 0, # no harmful algal bloom detected in region
        "HAB-present" : 1, # harmful algal bloom detected in region
}


In [30]:
tiff_paths = loadDataSet(path)


In [31]:
image_tensors = convertImages(tiff_paths)

c:\Users\chazd\AppData\Local\Programs\Python\Python39\lib\site-packages\rasterio\__init__.py:331: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [33]:
len(image_tensors)

30

In [34]:
image_labels = np.ones(len(image_tensors))

## Preprocessing

Before prepocessing the images, a quick review of the dataset

In [35]:
print("Dimensions: ", image_tensors[0].shape)
print("Batch Size: ", len(image_tensors))

Dimensions:  (424, 146, 4)
Batch Size:  30


In [36]:
# normalize all values
import cv2
import numpy as np

def normalize_4channel_image(image):
    # Split the image into Red, Green, Blue, and NIR channels
    r, g, b, nir = cv2.split(image)

    # Normalize each channel
    r_normalized = (r - np.min(r)) / (np.max(r) - np.min(r))
    g_normalized = (g - np.min(g)) / (np.max(g) - np.min(g))
    b_normalized = (b - np.min(b)) / (np.max(b) - np.min(b))
    nir_normalized = (nir - np.min(nir)) / (np.max(nir) - np.min(nir))

    # Recombine the channels
    normalized_image = cv2.merge([r_normalized, g_normalized, b_normalized, nir_normalized])

    return normalized_image



In [37]:
normalized= image_tensors

for i in range(len(image_tensors)):
    
    normalized[i] = normalize_4channel_image(image_tensors[i])
    


In [40]:
#split into test and train to 80:20
split = np.random.randint(0,len(normalized), size = int(len(normalized)*0.8))

train_images = normalized[split]
train_labels = image_labels[split]
test_images = normalized[-split]
test_labels = image_labels[-split]

TypeError: only integer scalar arrays can be converted to a scalar index

In [48]:
# Split the data into train and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    normalized, image_labels, test_size=0.2, random_state=42
)

## Building Model

In [50]:
from tensorflow.keras import datasets, layers, models

In [52]:
model = models.Sequential()

model.add(layers.Conv2D(16,(3,3),1,activation= 'softmax', input_shape=(160,175,4))),
model.add(layers.MaxPooling2D()),

model.add(layers.Conv2D(32,(3,3),1, activation= 'softmax')),
model.add(layers.MaxPooling2D()),

model.add(layers.Conv2D(16,(3,3),1, activation= 'softmax')),
model.add(layers.MaxPooling2D()),

# ry softmax

model.add(layers.Flatten()),
model.add(layers.Dense(16, activation='softmax')),
model.add(layers.Dense(1, activation='sigmoid')) 


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 158, 173, 16)      592       
                                                                 
 max_pooling2d (MaxPooling2  (None, 79, 86, 16)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 77, 84, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 38, 42, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 36, 40, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 18, 20, 16)        

In [54]:
#compile
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy', 'precision', 'recall', 'F1Score', 'AUC '])


## Train Model

In [23]:
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

Epoch 1/10


c:\Users\chazd\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2/2 [==============================] - 5s 1s/step - loss: 1.5231 - accuracy: 0.2034 - val_loss: 1.4345 - val_accuracy: 0.1525
Epoch 2/10
2/2 [==============================] - 1s 310ms/step - loss: 1.3100 - accuracy: 0.5424 - val_loss: 1.2552 - val_accuracy: 0.6102
Epoch 3/10
2/2 [==============================] - 0s 271ms/step - loss: 1.2550 - accuracy: 0.5424 - val_loss: 1.1626 - val_accuracy: 0.6102
Epoch 4/10
2/2 [==============================] - 0s 275ms/step - loss: 1.2007 - accuracy: 0.5424 - val_loss: 1.1490 - val_accuracy: 0.6102
Epoch 5/10
2/2 [==============================] - 0s 292ms/step - loss: 1.1278 - accuracy: 0.5424 - val_loss: 1.0990 - val_accuracy: 0.6102
Epoch 6/10
2/2 [==============================] - 0s 271ms/step - loss: 1.0620 - accuracy: 0.5424 - val_loss: 1.0531 - val_accuracy: 0.6102
Epoch 7/10
2/2 [==============================] - 0s 274ms/step - loss: 1.0067 - accuracy: 0.5424 - val_loss: 0.9739 - val_accuracy: 0.6102
Epoch 8/10
2/2 [==================

## Model Evaluation

In [24]:
import matplotlib.pyplot as plt

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)


2/2 - 0s - loss: 0.8584 - accuracy: 0.6780 - 113ms/epoch - 57ms/step


In [25]:
print("Accuracy = ", test_acc*100, "%")

Accuracy =  67.79661178588867 %
